# 🧬 Training pipeline: Training retrieval model

In this notebook, you will train a retrieval model that will be able to quickly generate a small subset of candidate items from a large collection of items. Your model will be based on the *two-tower architecture*, which embeds queries and candidates (keys) into a shared low-dimensional vector space. Here, a query consists of features of a customer and a transaction (e.g. timestamp of the purchase), whereas a candidate consists of features of a particular item. All queries will have a user ID and all candidates will have an item ID, and the model will be trained such that the embedding of a user will be close to all the embeddings of items the user has previously bought.

After training the model you will save its components to your local.

Let's go ahead and load the data.

## 📝 Imports

In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings
import polars as pl

warnings.filterwarnings("ignore")

from loguru import logger

from recsys.config import settings
from recsys.core.models import two_tower
from recsys.gcp.feature_store import datasets
from recsys.gcp.vertex_ai.serving.two_tower import GCPQueryModel, GCPCandidateModel

## 💿 Create training dataset
You will train your retrieval model with a subset of features.

For the query embedding you will use:
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

For the candidate embedding you will use:
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [2]:
path = os.getcwd()[:-9]
fullpath = os.path.join(path, 'data/preprocessed')

In [3]:
trans_df = pl.read_csv(f'{fullpath}/transactions.csv')
articles_df = pl.read_parquet(f'{fullpath}/articles.parquet')
customers_df = pl.read_csv(f'{fullpath}/customers.csv')

In [ ]:
training_data = datasets.create_training_dataset(trans_df, articles_df, customers_df)

In [ ]:
training_data.head()

In [6]:
dataset = two_tower.TwoTowerDataset(
    training_data=training_data, batch_size=settings.TWO_TOWER_MODEL_BATCH_SIZE
)

In [ ]:
train_ds, val_ds = dataset.get_train_val_split()

Let's take a look at our dataset:

In [ ]:
logger.info(f"Training samples: {len(dataset.properties['train_df']):,}")
logger.info(f"Validation samples: {len(dataset.properties['val_df']):,}")

logger.info(f"Number of users: {len(dataset.properties['user_ids']):,}")
logger.info(f"Number of items: {len(dataset.properties['item_ids']):,}")

In [ ]:
dataset.properties["train_df"].head()

## 🗼🗼 Build the Two Tower model

The two tower model consist of two models:
- **Query model**: Generates a query representation of a given user and transaction features.
- **Candidate model**: Generates an item representation given item features.

**Both models produce embeddings that live in the same embedding space**. You let this space be low-dimensional to prevent overfitting on the training data. (Otherwise, the model might simply memorize previous purchases, which makes it recommend items customers already have bought).

You start with creating the query model.

In [10]:
query_model_factory = two_tower.QueryTowerFactory(dataset=dataset)
query_model = query_model_factory.build()

In [ ]:
query_model

The candidate model is very similar to the query model. A difference is that it has two categorical features as input, which you one-hot encode.

In [12]:
item_model_factory = two_tower.ItemTowerFactory(dataset=dataset)
item_model = item_model_factory.build()

You will evaluate the two tower model using the *top-100 accuracy*. That is, for each transaction in the validation data you will generate the associated query embedding and retrieve the set of the 100 items that are closest to this query in the embedding space. The top-100 accuracy measures how often the item that was actually bought is part of this subset. To evaluate this, you create a dataset of all unique items in the training data.

Create a TwoTowerModel with the specified **query_model** and **item_model**:

In [13]:
model_factory = two_tower.TwoTowerFactory(dataset=dataset)
model = model_factory.build(query_model=query_model, item_model=item_model)

### Training the model

In [ ]:
trainer = two_tower.TwoTowerTrainer(dataset=dataset, model=model)
history = trainer.train(train_ds, val_ds)

Let's take a look at the training and validation loss:

In [ ]:
import matplotlib.pyplot as plt

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

# Training loss subplot
ax1.plot(history.history["loss"], label="Training Loss", color="blue")
ax1.set_title("Training Loss Over Time")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()
ax1.grid(True)

# Validation loss subplot
ax2.plot(history.history["val_loss"], label="Validation Loss", color="red")
ax2.set_title("Validation Loss Over Time")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")
ax2.legend()
ax2.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()  # Uncomment to show the plot

# 🗄️ Save models to local

In [16]:
query_model_gcp = GCPQueryModel(model=query_model)
item_model_gcp = GCPCandidateModel(model=item_model)

In [ ]:
query_local_path = query_model_gcp.save_to_local("query_model")
item_model_path = item_model_gcp.save_to_local("item_model")